<a href="https://colab.research.google.com/github/anicelysantos/book-python-para-analise-de-dados/blob/main/agregacao_operacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
import pandas as pd
import numpy as np

# Funcionamento do GroupBy

In [3]:
df = pd.DataFrame({'key1':['a','a','b','b','a'], 'key':['one','two','one','two','one'], 'data1': np.random.randn(5), 'data2': np.random.randn(5)})
df

,key1,key,data1,data2
0,a,one,-0.515352,-1.671199
1,a,two,-0.945383,-1.642781
2,b,one,0.790056,0.317845
3,b,two,-0.893911,0.137386
4,a,one,0.163645,-0.526231


In [4]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [5]:
grouped.mean()

key1
a   -0.432363
b   -0.051928
Name: data1, dtype: float64

In [9]:
means = df['data1'].groupby([df['key1'], df['key']]).mean()
means

key1  key
a     one   -0.175853
      two   -0.945383
b     one    0.790056
      two   -0.893911
Name: data1, dtype: float64

In [11]:
means.unstack()

key,one,two
key1,,
a,-0.175853,-0.945383
b,0.790056,-0.893911


In [12]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005   -0.945383
            2006    0.790056
Ohio        2005   -0.704632
            2006    0.163645
Name: data1, dtype: float64

In [13]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.432363,-1.280070
b,-0.051928,0.227616


In [15]:
df.groupby(['key1', 'key']).mean()

data1     data2
key1 key                    
a    one -0.175853 -1.098715
     two -0.945383 -1.642781
b    one  0.790056  0.317845
     two -0.893911  0.137386

In [16]:
df.groupby(['key1', 'key']).size()

key1  key
a     one    2
      two    1
b     one    1
      two    1
dtype: int64

**Iterando por grupos**

In [17]:
for nome, grupo in df.groupby('key1'):
  print(nome)
  print(grupo)

a
  key1  key     data1     data2
0    a  one -0.515352 -1.671199
1    a  two -0.945383 -1.642781
4    a  one  0.163645 -0.526231
b
  key1  key     data1     data2
2    b  one  0.790056  0.317845
3    b  two -0.893911  0.137386


In [20]:
for (k1,k2), grupo in df.groupby(['key1','key']):
  print((k1,k2))
  print(grupo)

('a', 'one')
  key1  key     data1     data2
0    a  one -0.515352 -1.671199
4    a  one  0.163645 -0.526231
('a', 'two')
  key1  key     data1     data2
1    a  two -0.945383 -1.642781
('b', 'one')
  key1  key     data1     data2
2    b  one  0.790056  0.317845
('b', 'two')
  key1  key     data1     data2
3    b  two -0.893911  0.137386


In [21]:
#gerando dicionario com uma parte dos dados

parte = dict(list(df.groupby('key1')))
parte['b']

,key1,key,data1,data2
2,b,one,0.790056,0.317845
3,b,two,-0.893911,0.137386


In [22]:
df.dtypes

key1      object
key       object
data1    float64
data2    float64
dtype: object

In [25]:
grouped = df.groupby(df.dtypes, axis=1)
for dtype, grupo in grouped:
  print(dtype)
  print(grupo)

float64
      data1     data2
0 -0.515352 -1.671199
1 -0.945383 -1.642781
2  0.790056  0.317845
3 -0.893911  0.137386
4  0.163645 -0.526231
object
  key1  key
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


**Selecionando uma coluna ou um subconjunto de colunas**